In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
print(torch.__version__)
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn

from IPython.display import clear_output

import helper

0.4.1


# Batch Gradient Descent and Binary Classification

In [30]:
data_dir = '/Users/akshaykulkarni/flowers/train/'
batch_size = 1024
tf = transforms.Compose([transforms.Resize((255, 255)),
                         transforms.ToTensor(),
                        ])

train_dataset = datasets.ImageFolder(data_dir, transform = tf)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True, drop_last = True)

data_dir = '/Users/akshaykulkarni/flowers/test/'

test_dataset = datasets.ImageFolder(data_dir, transform = tf)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 32, shuffle = True, drop_last = True)

In [31]:
# Loading all the images in the single batch and flattening
images, labels = next(iter(train_loader))
print(images.shape)
images = images.view(-1, images.shape[0]).float()
print(images.shape)
images /= 255
labels = labels.float()

torch.Size([1024, 3, 255, 255])
torch.Size([195075, 1024])


### Initializing the parameters using no. of nodes and no. of layers

In [32]:
# no. of layers
layers = 3
# no. of nodes in each layer in a list including input and output layers
num_nodes = [images.shape[0], 512, 1]

# initializing weights according to no. of layers and no. of nodes
assert(layers == len(num_nodes))
W = {}
b = {}
for i in range(1, layers):
    var_name = str(i)
    W[var_name] = torch.empty(num_nodes[i - 1], num_nodes[i])
    # Xavier initialization of weights
    nn.init.xavier_uniform_(W[var_name], gain=nn.init.calculate_gain('sigmoid'))
    b[var_name] = torch.randn(num_nodes[i], 1)

In [33]:
def binary_cross_entropy(A, Y):
    loss = (-1 / batch_size) * (torch.matmul(torch.log(A), Y) + torch.matmul(torch.log(1 - A), (1 - Y)))
    return loss

def sigmoid(x):
    return (1 / (1 + torch.exp(-x)))

In [38]:
def forward(x):
    out = x
    for i in range(1, layers):
        var_name = str(i)
        out = torch.add(torch.matmul(W[var_name].t(), out), b[var_name])
        out = sigmoid(out)
        print(out.shape)
        
    return out

In [39]:
# out = forward(images)
# print(out)

torch.Size([512, 1024])
torch.Size([1, 1024])
tensor([[0.7255, 0.7255, 0.7255,  ..., 0.7254, 0.7254, 0.7254]])
